<a href="https://colab.research.google.com/github/KhawajaAbaid/medium-articles/blob/main/Optimizing_Weights_of_an_Weighted_Average_Ensemble_using_Optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction:
Taking weighted average can result in imporved score but the problem arises when deciding what weight to use for which model. Even if we do manage to choose some weights that improve our score in comparison to the simple average, how do we know these are the optimal weights? This is where this article comes in, using the very easy, effective and fast hyperparameter framework like optuna to find the optimal values for our weights, that result in the best score possible.


In this notebook, we'll introduce a new technique of finding optimized weights for combining multiple models into an ensemble using a popular hyperparameters tuning library called Optuna.

Dataset: California Housing Price

Task: Regression

Models: Ridge, XGBoost, LightGBM

# Installing and Importing required libraries — for our tutorial.

In [ ]:
!pip install optuna

In [12]:
# The essential libraries to work with data
import pandas as pd
import numpy as np

# Machine learning models that we'll use
import xgboost as xgb
import lightgbm as lgbm
from sklearn.linear_model import Ridge

# Dataset
from sklearn.datasets import fetch_california_housing

# optuna
import optuna

# We'll use MSE as our metric
from sklearn.metrics import mean_squared_error

# Fetching the data

In [13]:
# We set as_frame parameter to True and access the return object's "frame"
# attribute to get the dataset as pandas dataframe.

df = fetch_california_housing(as_frame=True)["frame"]

df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [14]:
len(df)

20640

# Some Preprocessing

MedHouseVal is our target variable — the variable that we'll predict based on the rest of the columns/features.

But first, we need to check for any missing values in the dataset and fill them.

In [15]:
df.isnull().sum()

MedInc         0
HouseAge       0
AveRooms       0
AveBedrms      0
Population     0
AveOccup       0
Latitude       0
Longitude      0
MedHouseVal    0
dtype: int64

Well looks like we're lucky to have no missing values. So get our data ready for training.

First let's store target values in a separete variable. And then drop the target column from our dataset.

In [16]:
y = df.MedHouseVal
df = df.drop(columns=["MedHouseVal"])

Let's split our dataset into a train and validation set. For that we'll use scikit-learn's train_test_split method.

In [17]:
from sklearn.model_selection import train_test_split

# we'll use a validation size of 20% or 0.20
X_train, X_val, y_train, y_val = train_test_split(df, y,
                                                  test_size=0.2,
                                                  shuffle=True,
                                                  random_state=1337)

# Setting a Baseline

Before we optimize the weights, let's first set a baseline by taking simple average over all model's predicitons.

In [18]:
# instantiating the model
ridge_model = Ridge()
# training the model
ridge_model.fit(X_train, y_train)
# predicting using the trained model
y_pred_ridge = ridge_model.predict(X_val)

xgb_model = xgb.XGBRegressor(objective="reg:squarederror")
xgb_model.fit(X_train, y_train, verbose=False)
y_pred_xgb = xgb_model.predict(X_val)

lgbm_model = lgbm.LGBMRegressor()
lgbm_model.fit(X_train, y_train, verbose=-1)
y_pred_lgbm = lgbm_model.predict(X_val)

# combining predictions by taking simple average using numpy
y_pred_final = np.mean([y_pred_ridge, y_pred_xgb, y_pred_lgbm], axis=0)

# let's calculate mse
mse = mean_squared_error(y_val, y_pred_final)

print(f"Simple Average Ensemble's MSE: {mse}")

Simple Average Ensemble's MSE: 0.26770854520705767


# A gentle introduction to Optuna:
Optuna is an automatic hyperparameter optimization software framework, particularly designed for machine learning. By default it uses a Bayesian optimizing algorithm (TPE) which is way faster than using somehting like GridSearchCV which checks for each set of hyperparameters values defining the search space. The reason behind why Bayesian optimization is faster than grid search is that it uses information from the previous iterations of search to choose future values instead of treating each set of hyperparameters independently.

Due to optuna's high modularity, we can use it for our task of finding optimal weights for taking a weighted average, and much more.


# How Optuna works?
Optuna requries us to define an objective function, which
defines the values ranges for each hyperparameter that we wish to tune aka
optimize
trains the model using hyperparameters values that it chooses from that range predicts the values for the validation set
computes the metric/score for those predictions
and finally returns a score or metric that depending on the metric, we intend to maximize or minimize.

We then create a new study, passing it direction parameter, call its optimize function passing it the objective function and number of trials we wish to run. Number of trials is how many times the study object will run the objective function, get the score value. Then depending on the return value and on the direction value (maximize or minimize), it will choose the next set of values for hyperparamters that it infers will achieve a better score.

Okay enough with the theory, let's get our hands dirty with implementation!

# Optimizing weights with Optuna

In [23]:
def objective(trial):
  STEP_SIZE = 10

  weights = []
  all_models_predictions = []

  # we'll use a variable for setting upper limit for suggested value
  # since we intend to update it after each weight suggestion
  upper_limit = 100

  w_ridge = trial.suggest_int("w_ridge", 0, upper_limit, step=STEP_SIZE)
  weights.append(w_ridge)

  # Update upper limit to 100 - all the previous weights combined, which in this case is just w_ridge
  # WHY? well because we want to keep our sum of all weights equal to 100
  # and this is one way of ensuring that!
  upper_limit -= sum(weights)
  upper_limit = upper_limit

  w_xgb = trial.suggest_int("w_xgb", 0, upper_limit, step=STEP_SIZE)
  weights.append(w_xgb)

  # for the final weight we won't use optuna, rather we'll manually set it equal
  # to whatever value remains after subtracting the sum of suggested weight values from 100
  # This will also make sure that the sum of all weights remains equal to 100.
  w_lgbm = 100 - sum(weights)
  weights.append(w_lgbm)

  # Just as a sanity check, we'll check that the sum of all weights is equal to 100
  weights_sum = sum(weights)

  if weights_sum != 100:
    raise Exception(f"Weights sum must be equal to 100. Instead {weights_sum} was encountered!")

  
  # We'll use the default parameter values for all our models
  
  ridge_model = Ridge()
  ridge_model.fit(X_train, y_train)
  y_pred_ridge = ridge_model.predict(X_val)
  all_models_predictions.append(y_pred_ridge)

  xgb_model = xgb.XGBRegressor(objective ='reg:squarederror')
  xgb_model.fit(X_train, y_train, verbose=False)
  y_pred_xgb = xgb_model.predict(X_val)
  all_models_predictions.append(y_pred_xgb)

  lgbm_model = lgbm.LGBMRegressor()
  lgbm_model.fit(X_train, y_train, verbose=-1)
  y_pred_lgbm = lgbm_model.predict(X_val)
  all_models_predictions.append(y_pred_lgbm)


  # let's take the weighted average of the predictions using numpy
  y_pred_final = np.average(all_models_predictions, weights=weights, axis=0)

  # computing our metric i.e. MSE
  mse = mean_squared_error(y_val, y_pred_final)

  return mse

In case you're wondering why are we passing the same variable name as argument to suggest_int function, it is because it's a way of telling optuna to store the parameter value against that name — basically using it as a key, and storing that key value pair inside its memory. For instance, when you'd access your optimized values after the study has completed, optuna will return a dictionary of key value pairs, with keys being the names you specified in the suggest function argument, and values being the optimized values of hyperparameters.

It is important to remember that all the names you pass to functions should be unique!

In [28]:
study = optuna.create_study(study_name="optimizing weights", direction="minimize")
study.optimize(objective, n_trials=20)

[I 2023-01-19 17:18:56,884] A new study created in memory with name: optimizing weights
[I 2023-01-19 17:18:58,344] Trial 0 finished with value: 0.2646131415415177 and parameters: {'w_ridge': 40, 'w_xgb': 10}. Best is trial 0 with value: 0.2646131415415177.
[I 2023-01-19 17:18:59,778] Trial 1 finished with value: 0.20799930576386458 and parameters: {'w_ridge': 0, 'w_xgb': 10}. Best is trial 1 with value: 0.20799930576386458.
[I 2023-01-19 17:19:01,198] Trial 2 finished with value: 0.3028275602304653 and parameters: {'w_ridge': 50, 'w_xgb': 20}. Best is trial 1 with value: 0.20799930576386458.
[I 2023-01-19 17:19:02,629] Trial 3 finished with value: 0.5081829880318993 and parameters: {'w_ridge': 100, 'w_xgb': 0}. Best is trial 1 with value: 0.20799930576386458.
[I 2023-01-19 17:19:04,050] Trial 4 finished with value: 0.2635175242461122 and parameters: {'w_ridge': 20, 'w_xgb': 60}. Best is trial 1 with value: 0.20799930576386458.
[I 2023-01-19 17:19:05,522] Trial 5 finished with value: 0

In [26]:
study.best_value

0.21000290127721108

In [ ]:
study.best_params

Our optimized weighted average has brought the mean square error to 0.21 in comparison to 0.26 which we got from taking simple average of the predictions of the same models.

So the advantage of optimizing weights for taking a weighted average looks pretty clear.


# Takeaways:
* Taking weighted average can significantly imporve our score when compared to the simple average, as it did in our case.
* Leveraging the high modularity of optuna, we can find optimal weights for our ensemble, very quickly and effectively.

Hope you found this article useful! You can find the notebook with all the code and comments on Github, as well as Colab. Since this is my first article, so I'd welcome any feedback as it would help me improve and produce better content. You can reach me on Twitter on in comments.